In [ ]:
## Importing packages

library(tidyverse) # metapackage with lots of helpful functions
library(data.table)
library(dplyr)
library(ggplot2)
library(stringr)
library(DT)
library(tidyr)
library(corrplot)
library(leaflet)
library(lubridate)
library(car)
library(lattice)
library(caret)
library(MASS)
library(broom)
library(ROCR)
library(gridExtra)

## Dataset

In this experiment, we used the training Heart Disease data set from the UCI Machine Learning Repository. The data was collected from several locations (Cleveland Clinic Foundation, Hungarian Institute of Cardiology, V.A. Medical Center, University Hospital Zurich).

In [ ]:
## Ensuring dataset is loaded
list.files(path = "../input")
heart_data <- fread('../input/heart-disease-uci/heart.csv')

# change to better column names
name <- c("age","sex","chest_pain","rest_bp","cholesterol","fasting_bloodsugar","rest_ecg","max_heartrate","excercise_angina","ST_depression","slope","n_major_vasel","thal","target")
names(heart_data) <- name

# params for plots
title.center <- theme(plot.title = element_text(hjust = 0.5))

## Variables in the dataset

* age: The person's age in years
* sex: The person's sex (1 = male, 0 = female)
* cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
* trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)
* chol: The person's cholesterol measurement in mg/dl
* fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
* restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
* thalach: The person's maximum heart rate achieved
* exang: Exercise induced angina (1 = yes; 0 = no)
* oldpeak: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot)
* slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
* ca: The number of major vessels (0-3)
* thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
* target: Heart disease (0 = no, 1 = yes)

In [ ]:
head(heart_data)

In [ ]:
cat_vars = c("sex", "chest_pain", "fasting_bloodsugar", "rest_ecg", "excercise_angina", "slope", "thal", "target", "n_major_vasel")
cont_vars = c("age", "rest_bp", "cholesterol", "max_heartrate", "ST_depression")

# converting all categorical variables to factors which are used for categorical data
for (var in cat_vars){
    heart_data[[var]] <- as.factor(heart_data[[var]])
}

In [ ]:
#observing the data at a glance
head(heart_data)
str(heart_data)
summary(heart_data)

In [ ]:
# 303 patients all with 13 attributes + 1 target variable
dim(heart_data)

# checking for missing values - None found
sum(is.na(heart_data))

## Flow of Data Analysis
* Descriptive statistics for target only!!! (Show countplot, maybe talk about imbalance or talk about heart disease rates)
* Descriptive statistics for cont vars split by target (Hist, Distribution plot, boxplot)
* Comment on outliers and normality (Could do some transformations to make it more normal)
* Descriptive statistics for cat vars split by target (Countplot)
* Comment on distribution
* Hypothesis + Testing (Use 2-way contingency table for categorical vars and T/Z-test for continuous variiables

### Possible Hypothesis
1. Sex affects probability of getting heart disease
2. thal affects probability of getting heart disease
3. exercise_angina affects probability of getting heart disease
4. Higher cholesterol increases probability of getting heart disease
5. Higher rest_bp increases probability of getting heart disease
6. ST_depression affects probability of getting heart disease

##### Hypothesis 1-3 uses cateogrical variables == 2-way contingency tables
##### Hypothesis 4-6 uses continuous variablres == Z/T Test

# Data Description of Target

In [ ]:
counts <- table(heart_data$target)
barplot(counts, main="Heart Disease Distribution",
   xlab="Presence of Heart Disease")

# Data Description of Continuous Vairiables with Target

1. all the code below need to duplicate for all continuous variables

In [ ]:
# boxplots of continuous variables split by target

par(mfrow=c(2,3))
boxplot(age~target, data=heart_data)
boxplot(rest_bp~target, data=heart_data)
boxplot(cholesterol~target, data=heart_data)
boxplot(max_heartrate~target, data=heart_data)
boxplot(ST_depression~target, data=heart_data)

In [ ]:
# violinplots of continuous variables split by target

require(gridExtra)
plot1 <- ggplot(heart_data, aes(x=target, y=age)) + geom_violin(trim=FALSE)
plot2 <- ggplot(heart_data, aes(x=target, y=rest_bp)) + geom_violin(trim=FALSE)
plot3 <- ggplot(heart_data, aes(x=target, y=cholesterol)) + geom_violin(trim=FALSE)
plot4 <- ggplot(heart_data, aes(x=target, y=max_heartrate)) + geom_violin(trim=FALSE)
plot5 <- ggplot(heart_data, aes(x=target, y=ST_depression)) + geom_violin(trim=FALSE)
grid.arrange(plot1, plot2, plot3, plot4, plot5, ncol=3)

In [ ]:
# plot_boxes <- function(cont_vars, data, target='target'){
#     len = length(cont_vars)
#     par(mfrow=c(2,ceiling(len/2)))
#     for (cont_var in cont_vars){
#         boxplot(data[[cont_var]], main =paste("Boxplot of", cont_var), width=100, height=6)
#     }
# }

# plot_ggs <- function(cont_vars, data){
#     p1 = ggplot(data, aes(x = data[[cont_vars[1]]])) + geom_density() + theme_bw() + theme_classic() + ggtitle(paste(cont_vars[1], "Distribution")) + ylab("Number of People")
#     p2 = ggplot(data, aes(x = data[[cont_vars[2]]])) + geom_density() + theme_bw() + theme_classic() + ggtitle(paste(cont_vars[2], "Distribution")) + ylab("Number of People")
#     p3 = ggplot(data, aes(x = data[[cont_vars[3]]])) + geom_density() + theme_bw() + theme_classic() + ggtitle(paste(cont_vars[3], "Distribution")) + ylab("Number of People")
#     p4 = ggplot(data, aes(x = data[[cont_vars[4]]])) + geom_density() + theme_bw() + theme_classic() + ggtitle(paste(cont_vars[4], "Distribution")) + ylab("Number of People")
#     p5 = ggplot(data, aes(x = data[[cont_vars[5]]])) + geom_density() + theme_bw() + theme_classic() + ggtitle(paste(cont_vars[5], "Distribution")) + ylab("Number of People")
#     len = length(cont_vars)
#     grid.arrange(p1, p2, p3, p4, p5, nrow=2)
# }

# plot_hists <- function(cont_vars, data){
#     len = length(cont_vars)
#     par(mfrow=c(2,ceiling(len/2)))
#     for (cont_var in cont_vars){
#         hist(data[[cont_var]], main =paste("Boxplot of", cont_var), breaks=50)
#     }
# }

# plot_ggs(cont_vars, heart_data)
# plot_boxes(cont_vars, heart_data)
# plot_hists(cont_vars, heart_data)

# Data Description of Categorical Vairiables with Target

1. all the code below need to duplicate for all categorical variables

In [ ]:
# need to figure out the legend

par(mfrow=c(3,3))

counts <- table(heart_data$target, heart_data$sex)
barplot(counts, main="Heart Disease Distribution by sex", xlab="sex")

counts <- table(heart_data$target, heart_data$chest_pain)
barplot(counts, main="Heart Disease Distribution by chest_pain", xlab="chest_pain")

counts <- table(heart_data$target, heart_data$fasting_bloodsugar)
barplot(counts, main="Heart Disease Distribution by fasting_bloodsugar", xlab="fasting_bloodsugar")

counts <- table(heart_data$target, heart_data$rest_ecg)
barplot(counts, main="Heart Disease Distribution by rest_ecg", xlab="rest_ecg")

counts <- table(heart_data$target, heart_data$excercise_angina)
barplot(counts, main="Heart Disease Distribution by excercise_angina", xlab="excercise_angina")

counts <- table(heart_data$target, heart_data$slope)
barplot(counts, main="Heart Disease Distribution by slope", xlab="slope")

counts <- table(heart_data$target, heart_data$thal)
barplot(counts, main="Heart Disease Distribution by thal", xlab="thal")

counts <- table(heart_data$target, heart_data$n_major_vasel)
barplot(counts, main="Heart Disease Distribution by n_major_vasel", xlab="n_major_vasel")

In [ ]:
# p1 = ggplot(heart_data,aes(factor(sex))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of sex") +title.center
# p2 = ggplot(heart_data,aes(factor(chest_pain))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of chest_pain") +title.center
# p3 = ggplot(heart_data,aes(factor(fasting_bloodsugar))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of fasting_bloodsugar") +title.center
# p4 = ggplot(heart_data,aes(factor(rest_ecg))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of rest_ecg") +title.center
# p5 = ggplot(heart_data,aes(factor(excercise_angina))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of excercise_angina") +title.center
# p6 = ggplot(heart_data,aes(factor(slope))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of slope") +title.center
# p7 = ggplot(heart_data,aes(factor(n_major_vasel))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of n_major_vasel") +title.center
# p8 = ggplot(heart_data,aes(factor(thal))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of thal") +title.center
# p9 = ggplot(heart_data,aes(factor(target))) + geom_bar(width = 0.2) + theme_bw() + theme_classic()+geom_text(stat ='count',aes(label =..count..),vjust =-0.2)+ggtitle("countplot of target") +title.center

# grid.arrange(p1, p2, p3, p4, p5, p6, p7, p8, p9, nrow=3)

# Section 3.2: Outliers & Robust Analysis
1. Continuous Variables (Compare standard rule and IQR rule, Use trimmed mean etc to see if there are outliers)
2. Categorical Variables (Look for categories which has only 1/a few instance)??

In [ ]:
# standard rule

print("Using standard rule")
mu = mean(heart_data$age)
sigma = sd(heart_data$age)
print(paste("There are", sum(abs(heart_data$age - mu) > 2*sigma), "outliers"))

mu = mean(heart_data$rest_bp)
sigma = sd(heart_data$rest_bp)
print(paste("There are", sum(abs(heart_data$rest_bp - mu) > 2*sigma), "outliers"))

mu = mean(heart_data$cholesterol)
sigma = sd(heart_data$cholesterol)
print(paste("There are", sum(abs(heart_data$cholesterol - mu) > 2*sigma), "outliers"))

mu = mean(heart_data$max_heartrate)
sigma = sd(heart_data$max_heartrate)
print(paste("There are", sum(abs(heart_data$max_heartrate - mu) > 2*sigma), "outliers"))

mu = mean(heart_data$ST_depression)
sigma = sd(heart_data$ST_depression)
print(paste("There are", sum(abs(heart_data$ST_depression - mu) > 2*sigma), "outliers"))


In [ ]:
# iqr rule

print("Using IQR rule")
iqr = IQR(heart_data$age)
q1 = quantile(heart_data$age, 0.25)
q3 = quantile(heart_data$age, 0.75)
lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr
print(paste("There are", sum(heart_data$age > upper_bound | heart_data$age < lower_bound), "outliers"))

iqr = IQR(heart_data$rest_bp)
q1 = quantile(heart_data$rest_bp, 0.25)
q3 = quantile(heart_data$rest_bp, 0.75)
lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr
print(paste("There are", sum(heart_data$rest_bp > upper_bound | heart_data$rest_bp < lower_bound), "outliers"))

iqr = IQR(heart_data$cholesterol)
q1 = quantile(heart_data$cholesterol, 0.25)
q3 = quantile(heart_data$cholesterol, 0.75)
lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr
print(paste("There are", sum(heart_data$cholesterol > upper_bound | heart_data$cholesterol < lower_bound), "outliers"))

iqr = IQR(heart_data$max_heartrate)
q1 = quantile(heart_data$max_heartrate, 0.25)
q3 = quantile(heart_data$max_heartrate, 0.75)
lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr
print(paste("There are", sum(heart_data$max_heartrate > upper_bound | heart_data$max_heartrate < lower_bound), "outliers"))

iqr = IQR(heart_data$ST_depression)
q1 = quantile(heart_data$ST_depression, 0.25)
q3 = quantile(heart_data$ST_depression, 0.75)
lower_bound = q1-1.5*iqr
upper_bound = q3+1.5*iqr
print(paste("There are", sum(heart_data$ST_depression > upper_bound | heart_data$ST_depression < lower_bound), "outliers"))

In [ ]:
# find out percentage change when creating report

print(paste("Mean of age:",mean(heart_data$age)))
print(paste("Trimmed Mean (0.1) of age:",mean(heart_data$age, trim=0.1)))
print(paste("Trimmed Mean (0.2) of age:",mean(heart_data$age, trim=0.2)))

print(paste("Mean of rest_bp:",mean(heart_data$rest_bp)))
print(paste("Trimmed Mean (0.1) of rest_bp:",mean(heart_data$rest_bp, trim=0.1)))
print(paste("Trimmed Mean (0.2) of rest_bp:",mean(heart_data$rest_bp, trim=0.2)))

print(paste("Mean of cholesterol:",mean(heart_data$cholesterol)))
print(paste("Trimmed Mean (0.1) of cholesterol:",mean(heart_data$cholesterol, trim=0.1)))
print(paste("Trimmed Mean (0.2) of cholesterol:",mean(heart_data$cholesterol, trim=0.2)))

print(paste("Mean of max_heartrate:",mean(heart_data$max_heartrate)))
print(paste("Trimmed Mean (0.1) of max_heartrate:",mean(heart_data$max_heartrate, trim=0.1)))
print(paste("Trimmed Mean (0.2) of max_heartrate:",mean(heart_data$max_heartrate, trim=0.2)))

print(paste("Mean of ST_depression:",mean(heart_data$ST_depression)))
print(paste("Trimmed Mean (0.1) of ST_depression:",mean(heart_data$ST_depression, trim=0.1)))
print(paste("Trimmed Mean (0.2) of ST_depression:",mean(heart_data$ST_depression, trim=0.2)))

In [ ]:
cat_vars

In [ ]:
# 3 groups of outliers
# rest_ecg == 2 only has 4 instances
# thal == 0 only has 2 instances
# n_major_vasel == 4 only has 5 instances

print("sex")
summary(heart_data$sex)
print("chest_pain")
summary(heart_data$chest_pain)
print("fasting_bloodsugar")
summary(heart_data$fasting_bloodsugar)
print("rest_ecg")
summary(heart_data$rest_ecg)
print("excercise_angina")
summary(heart_data$excercise_angina)
print("slope")
summary(heart_data$slope)
print("thal")
summary(heart_data$thal)
print("n_major_vasel")
summary(heart_data$n_major_vasel)

# Section 3.3: Categorical analysis (Association tests)
1. Associations between continuous variables and target
2. Associations between categorical variables and target

In [ ]:
# H0 - No association between cat_var with target
# H1 - Association between cat_var with target

association_test <- function(counts){
    colsum = matrix(colSums(counts), ncol=dim(counts)[-1])
    rowsum = matrix(rowSums(counts), ncol=1)
    
    #print(colsum)
    #print(rowsum)
    expected = rowsum %*% colsum / sum(colsum)
    # print(expected)
    chisq = sum((counts-expected)^2 / expected)
    
    v = (nrow(counts)-1) * (ncol(counts)-1)
    pvalue = 1 - pchisq(chisq, df=v)
    return (pvalue)
}

In [ ]:
library(dvmisc)

# split continuous variables into 4 groups using quant_group
print(paste("P-value between age and target:", association_test(table(heart_data$target, quant_groups(heart_data$age)))))
print(paste("P-value between rest_bp and target:", association_test(table(heart_data$target, quant_groups(heart_data$rest_bp)))))
print(paste("P-value between cholesterol and target:", association_test(table(heart_data$target, quant_groups(heart_data$cholesterol)))))
print(paste("P-value between max_heartrate and target:", association_test(table(heart_data$target, quant_groups(heart_data$max_heartrate)))))
print(paste("P-value between ST_depression and target:", association_test(table(heart_data$target, quant_groups(heart_data$ST_depression, groups=3)))))

In [ ]:
print(paste("P-value between sex and target:", association_test(table(heart_data$target, heart_data$sex))))
print(paste("P-value between chest_pain and target:", association_test(table(heart_data$target, heart_data$chest_pain))))
print(paste("P-value between fasting_bloodsugar and target:", association_test(table(heart_data$target, heart_data$fasting_bloodsugar))))
print(paste("P-value between rest_ecg and target:", association_test(table(heart_data$target, heart_data$rest_ecg))))
print(paste("P-value between excercise_angina and target:", association_test(table(heart_data$target, heart_data$excercise_angina))))
print(paste("P-value between slope and target:", association_test(table(heart_data$target, heart_data$slope))))
print(paste("P-value between thal and target:", association_test(table(heart_data$target, heart_data$thal))))
print(paste("P-value between n_major_vasel and target:", association_test(table(heart_data$target, heart_data$n_major_vasel))))

# Section 3.4: Hypothesis Testing
1. Hypothesis Testing (Whether variable X affects target)
2. ANOVA to check if continuous variables ditribution differs for target=0/1 (Checks if mean is equal), not suitable in our case as it is used for >2 populations
3. Wilcoxon Rank Sum Test to check if continuous variables ditribution differs for target=0/1 (Checks if distribution is identical)

In [ ]:
# Section 3.4.1: Sample test - Hypothesis testing

# variance to test to check if variance of both population are equal
# H0: Variance equal
# H1: Variance not equal

# t-test to see if mean of 2 populations are identical
# H0: Means are identical
# H1: Means are not identical
population1 = subset(heart_data, target==0)$age
population2 = subset(heart_data, target==1)$age
var.test(population1, population2) # variance not equal
t.test(population1, population2, var.equal=FALSE)

population1 = subset(heart_data, target==0)$rest_bp
population2 = subset(heart_data, target==1)$rest_bp
var.test(population1, population2) # variance not equal
t.test(population1, population2, var.equal=FALSE)

population1 = subset(heart_data, target==0)$cholesterol
population2 = subset(heart_data, target==1)$cholesterol
var.test(population1, population2) # variance not equal
t.test(population1, population2, var.equal=FALSE)

population1 = subset(heart_data, target==0)$max_heartrate
population2 = subset(heart_data, target==1)$max_heartrate
var.test(population1, population2) # variance not equal
t.test(population1, population2, var.equal=FALSE)

population1 = subset(heart_data, target==0)$ST_depression
population2 = subset(heart_data, target==1)$ST_depression
var.test(population1, population2) # variance not equal
t.test(population1, population2, var.equal=FALSE)

In [ ]:
# Section 3.4.2: ANOVA

# continiuous variables ONLY
# summary(aov(heart_data$age~factor(heart_data$target)))
# summary(aov(heart_data$rest_bp~factor(heart_data$target)))
# summary(aov(heart_data$cholesterol~factor(heart_data$target)))
# summary(aov(heart_data$max_heartrate~factor(heart_data$target)))
# summary(aov(heart_data$ST_depression~factor(heart_data$target)))

In [ ]:
# Section 3.4.3: Wilcoxon Rank Sum Test
# H0: Distribution of variable for target=0 and target=1 are identical
# H1: Distribution of variable for target=0 and target=1 are not identical

wilcox.test(age~target, data=heart_data)
wilcox.test(rest_bp~target, data=heart_data)
wilcox.test(cholesterol~target, data=heart_data)
wilcox.test(max_heartrate~target, data=heart_data)
wilcox.test(ST_depression~target, data=heart_data)

# Section 3.5: Correlation & Regression
1. Correlation Heatmap
2. Multiple Linear Regression

In [ ]:
# treat target as a continuous value i.e. Probability of having heart disease

heart_data$target <- as.numeric(heart_data$target)

In [ ]:
install.packages("corrplot")
source("http://www.sthda.com/upload/rquery_cormat.r")

In [ ]:
# heatmaps
cont_values <- subset(heart_data, select = c(1, 4, 5, 8, 10, 14))
rquery.cormat(cont_values, type="full")

In [ ]:
cor(cont_values)

In [ ]:
# multiple linear regression
model <- lm(target ~ age + rest_bp + cholesterol + max_heartrate + ST_depression, data = cont_values)
summary(model)